<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Modeling_ICG_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
!pip install catboost > log.txt
import autorch
import random
torch.manual_seed(11)
np.random.seed(11)
random.seed(11)

In [2]:
def get_col(df,name):
  return df.columns[df.columns.str.contains(name)]

![img](https://i.imgur.com/hksoTrh.jpg)



# load data

In [3]:
c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c.keys(),c['x'],c['y'],len(c['x']),len(c['y'])

(dict_keys(['x', 'y']),
 ['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from V615 Btm    Toluene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from C820 Dist    Toluene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr',
  'Simulation Case Conditions_Feed Rate_Feed from T651    NA_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651    Benzene_wt%',
  'Simulation Case Conditions_Feed Rate_Feed from T651    Toluene_wt%',
  'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%',
  'Simulation Case Conditions_Spec 2 : NA in Ben

In [4]:
df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
df.head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
001-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,980.0,0.5
001-005,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,950.0,1.0
001-008,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,920.0,1.5
001-011,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,980.0,0.5
001-014,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,950.0,1.0


# Distillate Rate_m3/hr == 0.01的部份 改成 0

In [5]:
df.loc[df['Simulation Case Conditions_C620 Distillate Rate_m3/hr']==0.01,'Simulation Case Conditions_C620 Distillate Rate_m3/hr'] = 0

In [6]:
df[c['y']].max(),df[c['y']].min()

(Simulation Case Conditions_C620 Distillate Rate_m3/hr    10.25
 dtype: float64, Simulation Case Conditions_C620 Distillate Rate_m3/hr    0.0
 dtype: float64)

# modeling 不要限制輸出 可以有負值

In [7]:
import catboost
from sklearn.model_selection import train_test_split
X = df[c['x']]
y = df[c['y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
c620_icg = catboost.CatBoostRegressor()
c620_icg.fit(X_train,y_train)

Learning rate set to 0.038059
0:	learn: 1.1947814	total: 46.8ms	remaining: 46.8s
1:	learn: 1.1677441	total: 47.2ms	remaining: 23.6s
2:	learn: 1.1422523	total: 47.7ms	remaining: 15.8s
3:	learn: 1.1137243	total: 48.1ms	remaining: 12s
4:	learn: 1.0870422	total: 48.5ms	remaining: 9.65s
5:	learn: 1.0585108	total: 48.9ms	remaining: 8.1s
6:	learn: 1.0315739	total: 49.3ms	remaining: 6.99s
7:	learn: 1.0042701	total: 49.7ms	remaining: 6.16s
8:	learn: 0.9849588	total: 50.1ms	remaining: 5.52s
9:	learn: 0.9618368	total: 50.5ms	remaining: 5s
10:	learn: 0.9383505	total: 51ms	remaining: 4.58s
11:	learn: 0.9175390	total: 51.4ms	remaining: 4.23s
12:	learn: 0.8943633	total: 51.9ms	remaining: 3.94s
13:	learn: 0.8704499	total: 52.3ms	remaining: 3.68s
14:	learn: 0.8490581	total: 52.8ms	remaining: 3.46s
15:	learn: 0.8291123	total: 53.2ms	remaining: 3.27s
16:	learn: 0.8116943	total: 53.7ms	remaining: 3.1s
17:	learn: 0.7912556	total: 54ms	remaining: 2.95s
18:	learn: 0.7769928	total: 54.4ms	remaining: 2.81s
19:

# evaluate

In [9]:
from sklearn.metrics import r2_score,mean_squared_error

def mape(y_true, y_pred, e = 1e-8):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true > e
    y_true, y_pred = y_true[mask], y_pred[mask]
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def show_metrics(y_real,y_pred,e=1e-8):
    res = pd.DataFrame(index=y_pred.columns,columns=['R2','MSE','MAPE'])
    for i in y_pred.columns:
        res.loc[i,'R2'] = np.clip(r2_score(y_real[i],y_pred[i]),0,1)
        res.loc[i,'MSE'] = mean_squared_error(y_real[i],y_pred[i])
        res.loc[i,'MAPE'] = mape(y_real[i],y_pred[i],e)
    res.loc['AVG'] = res.mean(axis=0)
    return res

In [10]:
y_real = y_test
y_pred = pd.DataFrame(c620_icg.predict(X_test),index=y_real.index,columns=y_real.columns)
show_metrics(y_real,y_pred)

,R2,MSE,MAPE
Simulation Case Conditions_C620 Distillate Rate_m3/hr,0.993625,0.0114553,11.791
AVG,0.993625,0.0114553,11.791


# ICG while 迴圈

In [11]:
def ICG(Input):
  while True:
    output = pd.DataFrame(c620_icg.predict(Input.values),index=Input.index,columns=['Simulation Case Conditions_C620 Distillate Rate_m3/hr'])
    dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
    na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    print('current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{}'.format(dist_rate,na_in_benzene))
    if output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0] > 0:
      return output,Input
    else:
      Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
      print('NA in Benzene_ppmw -= 30')

In [12]:
print(df.shape)
df = df[df['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']==70.0]
df = df[df['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']==980.0]
print(df.shape)

(1296, 15)
(122, 15)


In [13]:
for i in range(10):
  Input = df[c['x']].sample(1)
  output,Input = ICG(Input)
  print(i)

current Distillate Rate_m3/hr:0.00360208155530628 NA in Benzene_ppmw:980.0
0
current Distillate Rate_m3/hr:3.424096545691832 NA in Benzene_ppmw:980.0
1
current Distillate Rate_m3/hr:-0.06787439115421756 NA in Benzene_ppmw:980.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.04405851392676219 NA in Benzene_ppmw:950.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:0.025209156815565292 NA in Benzene_ppmw:920.0
2
current Distillate Rate_m3/hr:0.6449415114195205 NA in Benzene_ppmw:980.0
3
current Distillate Rate_m3/hr:0.26864910173997325 NA in Benzene_ppmw:980.0
4
current Distillate Rate_m3/hr:0.01411958558919818 NA in Benzene_ppmw:980.0
5
current Distillate Rate_m3/hr:5.687416854904736 NA in Benzene_ppmw:980.0
6
current Distillate Rate_m3/hr:0.8194682273884752 NA in Benzene_ppmw:980.0
7
current Distillate Rate_m3/hr:1.325996261843392 NA in Benzene_ppmw:980.0
8
current Distillate Rate_m3/hr:0.6449415114195205 NA in Benzene_ppmw:980.0
9


# 特別資料測試

In [14]:
Input = {
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr': {0: 183.71834522298502},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%': {0: 2.06},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%': {0: 11.91},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%': {0: 37.48},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr': {0: 18.599028347620738},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%': {0: 2.51},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%': {0: 69.02},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%': {0: 20.62},
'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr': {0: 103.76780384010259},
'Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%': {0: 0.685},
'Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%': {0: 46.628},
'Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%': {0: 40.868},
'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%': {0: 89.39},
'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': {0: 849.0}}
Input = pd.DataFrame(Input)
output,Input = ICG(Input)

current Distillate Rate_m3/hr:0.02212001934315505 NA in Benzene_ppmw:849.0


In [15]:
Input = {
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr': {0: 171.58942010720452},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%': {0: 1.971},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%': {0: 12.146},
'Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%': {0: 37.791},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr': {0: 19.669237218735567},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%': {0: 2.56},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%': {0: 68.421},
'Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%': {0: 21.33},
'Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr': {0: 101.95451869681709},
'Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%': {0: 0.777},
'Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%': {0: 50.531},
'Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%': {0: 35.435},
'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%': {0: 89.851},
'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': {0: 1193.0}}
Input = pd.DataFrame(Input)
output,Input = ICG(Input)

current Distillate Rate_m3/hr:0.0489690063644056 NA in Benzene_ppmw:1193.0


In [16]:
joblib.dump(c620_icg,'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_catboost.pkl')

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg_catboost.pkl']